# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import joblib

from azureml.widgets import RunDetails
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.pipeline.steps import AutoMLStep
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.datastore import Datastore


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
#assign work space
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'California-housing-price-projection'
experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FWPMXBRG4 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
quick-starts-ws-137963
aml-quickstarts-137963
southcentralus
9a7511b8-150f-4a58-8528-3e7d50216c31


In [3]:
#create a compute target
amlcompute_cluster_name = "compute-housing"

# Verify that cluster does not exist already, if not create a new one with amlcompute_cluster_name
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 0, timeout_in_minutes = 10)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

This dataset contains California Housing Price data downloaded from https://raw.githubusercontent.com/ageron/handson-ml/master/ and was used as an example in Hands-On Machine Learning with Scikit-Learn & TensorFlow by Aurelien Geron. 

This dataset is an adapted version from the original data from the StatLib repository collected from the 1990 California census. 

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
#check if the dataset exists and register the dataset if not
import train
found = False
key = "California_housing_price-1"
description_text = "California housing price from 1990 census"

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]
    print("found dataset") 

if not found:       
        #If the dataset doesn't exist, then download data and register the Dataset in Workspace
#        DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
#        HOUSING_PATH = "datasets/housing"
#        HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + "/housing.tgz"
#        df = train.fetch_housing_data(HOUSING_URL, HOUSING_PATH)
    path = r"https://github.com/second-husky/Capstone-Azure-ML/raw/master/starter_file/housing.csv"
    datastore = Datastore(ws, name = None)
    df = pd.read_csv(path).dropna()
#    print(df.head(5))
    dataset = TabularDatasetFactory.register_pandas_dataframe(df, datastore, 'housing_data')


df = dataset.to_pandas_dataframe()
df.describe()

datasets/housing/housing.csv
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/bd2e758d-4831-4d2a-a381-6ddc8964f22e/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "iterations" : 20,
    "primary_metric" : 'r2_score'
}
project_folder = "./housing"
# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "regression",
                             training_data=dataset,
                             label_column_name="median_house_value",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

Trained models include StackEnsemble, VotingEnsemble, XGBoostRegressor, LightGBM, RandomForest, etc. 
StackEnsemble and VotingEnsemble are the highest proforming models. 
And most models with high performance used XGBoostRegressor

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
exp_automl = Experiment(ws, 'AutoML-housing')
automl_run = exp_automl.submit(automl_config, show_output = False)
RunDetails(automl_run).show()
automl_run.wait_for_completion()

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_0d99916a-5d12-4288-924d-a245a9337929',
 'target': 'compute-housing',
 'status': 'Completed',
 'startTimeUtc': '2021-02-07T18:38:14.473308Z',
 'endTimeUtc': '2021-02-07T18:57:23.260737Z',
 'properties': {'num_iterations': '20',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-housing',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML-housing","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-137963","workspace_name":"quick-starts-ws-137963","region":"southcentralus","compute_target":"compute-housing","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":20,"primary_metric":"r2_score","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
#save the best model using joblib
best_automl_run, best_automl_model = automl_run.get_output()
os.makedirs('./outputs_automl', exist_ok = True)
ml_path = r'./outputs_automl/model_best.joblib'
joblib.dump(value = best_automl_model, filename = ml_path)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


['./outputs_automl/model_best.joblib']

In [8]:
print(best_automl_run)

Run(Experiment: AutoML-housing,
Id: AutoML_0d99916a-5d12-4288-924d-a245a9337929_18,
Type: azureml.scriptrun,
Status: Completed)


In [35]:
print(best_automl_model.steps[1][1])

PreFittedSoftVotingRegressor(estimators=[('4',
                                          Pipeline(memory=None,
                                                   steps=[('standardscalerwrapper',
                                                           <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7fe4b4a116a0>),
                                                          ('xgboostregressor',
                                                           XGBoostRegressor(base_score=0.5,
                                                                            booster='gbtree',
                                                                            colsample_bylevel=1,
                                                                            colsample_bynode=1,
                                                                            colsample_bytree=0.9,
                                                                            eta=0.5,
         

In [10]:
#register the best model using model name
model_name = best_automl_run.properties['model_name']
automl_model = automl_run.register_model(model_name = model_name)

In [11]:
best_automl_run.get_metrics()

{'median_absolute_error': 19905.220327147093,
 'root_mean_squared_log_error': 0.22225926932754933,
 'root_mean_squared_error': 46142.78083497149,
 'normalized_root_mean_squared_error': 0.09513936197164442,
 'spearman_correlation': 0.9277169375929726,
 'normalized_median_absolute_error': 0.04104152215278925,
 'normalized_root_mean_squared_log_error': 0.06338381464132324,
 'mean_absolute_percentage_error': 16.847619375684243,
 'explained_variance': 0.8455784812684746,
 'normalized_mean_absolute_error': 0.06286445353284707,
 'r2_score': 0.8455589515361349,
 'mean_absolute_error': 30489.385692337895,
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_0d99916a-5d12-4288-924d-a245a9337929_18/residuals',
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_0d99916a-5d12-4288-924d-a245a9337929_18/predicted_true'}

In [12]:
#display the properties of the best model
best_automl_run.get_properties()

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'regression\',\'primary_metric\':\'r2_score\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'AutoML-housing\',\'compute_target\':\'compute-housing\',\'subscription_id\':\'9a7511b8-150f-4a58-8528-3e7d50216c31\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_0d99916a-5d12-4288-924d-a245a9337929_18","experiment_name":"AutoML-housing","workspace_name":"quick-starts-ws-137963","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group_name":"aml-quickstarts-137963"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '18',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azureml.ComputeTargetTy

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [13]:
#check the files associated with best run
best_automl_run.get_file_names()

['automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_7e172b238385e78714939552da82e2b22ec06945079a243dd2caceb3a1fefe60_p.txt',
 'azureml-logs/65_job_prep-tvmps_7e172b238385e78714939552da82e2b22ec06945079a243dd2caceb3a1fefe60_p.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_7e172b238385e78714939552da82e2b22ec06945079a243dd2caceb3a1fefe60_p.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/102_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py',
 'predicted_true',
 'residuals']

In [14]:
#download environment and scoring script associated with the best run
best_automl_run.download_file('outputs/conda_env_v_1_0_0.yml')
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py')

In [22]:
#create an inference configuration
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.from_conda_specification("myenv", 'conda_env_v_1_0_0.yml')
#env = Environment.get(workspace = ws, name = "AzureML-AutoML")

for pip_package in ["pandas","scikit-learn","joblib"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='scoring_file_v_1_0_0.py',
                                    environment=env)

In [23]:
from azureml.core.webservice import AciWebservice, AksWebservice, LocalWebservice
from azureml.core.model import InferenceConfig, Model

#deployment_config = LocalWebservice.deploy_configuration(port = 9000)
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, enable_app_insights = True)
first_model = Model(ws, name = model_name)
service = Model.deploy(ws, "automl-service", [first_model], inference_config, deployment_config, overwrite = True)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..........................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
2021-02-07T20:10:46,658046166+00:00 - rsyslog/run 
2021-02-07T20:10:46,660187112+00:00 - iot-server/run 
2021-02-07T20:10:46,661962050+00:00 - nginx/run 
2021-02-07T20:10:46,661962150+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_eac1280dde495806310462165568faef/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_eac1280dde495806310462165568faef/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azu

TODO: In the cell below, send a request to the web service you deployed to test it.

In [24]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [25]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

data = {
    "data":
    [
        {
            'longitude': "-122.23",
            'latitude': "37.88",
            'housing_median_age': "41",
            'total_rooms': "880",
            'total_bedrooms': "129",
            'population': "322",
            'households': "126",
            'median_income': "8.3252",
            'ocean_proximity': "NEAR BAY",
        },
    ],
}

body = str.encode(json.dumps(data))

url = service.scoring_uri
api_key = '' 
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

b'"{\\"result\\": [459947.7060546025]}"'


TODO: In the cell below, print the logs of the web service and delete the service

In [26]:
service.get_logs()


'2021-02-07T20:10:46,658046166+00:00 - rsyslog/run \n2021-02-07T20:10:46,660187112+00:00 - iot-server/run \n2021-02-07T20:10:46,661962050+00:00 - nginx/run \n2021-02-07T20:10:46,661962150+00:00 - gunicorn/run \n/usr/sbin/nginx: /azureml-envs/azureml_eac1280dde495806310462165568faef/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_eac1280dde495806310462165568faef/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_eac1280dde495806310462165568faef/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_eac1280dde495806310462165568faef/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_eac1280dde495806310462165568faef/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [ ]:
service.delete()